In [1]:
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive/'Colab Notebooks'/RL_Python
!ls

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/RL_Python
baby-steps-of-rl-ja	    epsilon_TS_gamma.py  RL_Python.ipynb
bayesian_ucb.py		    experiment.py	 thompson_sampling.py
bayesian_ucb_TS_epsilon.py  Kaggle_bandit.ipynb  ucb_decay.py
bayesian_ucb_TS.py	    pull_same.py
epsilon_TS_beta.py	    pull_same_TS_ucb.py


In [ ]:
!cat ./baby-steps-of-rl-ja/EL/actor_critic.py

＊提出

＊実験

In [ ]:
!ls

sample_data


In [ ]:
%%writefile epsilon_TS_beta.py

import json
import numpy as np
import pandas as pd
import random

bandit_state = None
total_reward = 0
epsilon = 0.12
    
def epsilon_TS_beta_agent (observation, configuration):
    global history, history_bandit

    step = 1.0 #you can regulate exploration / exploitation balacne using this param
    decay_rate = 0.97 # how much do we decay the win count after each call
    
    global bandit_state,total_reward, epsilon

    if observation.step == 0:
        # initial bandit state
        bandit_state = [[1,1] for i in range(configuration["banditCount"])]
    else:       
        # updating bandit_state using the result of the previous step
        last_reward = observation["reward"] - total_reward
        total_reward = observation["reward"]
        
        # we need to understand who we are Player 1 or 2
        last_bandit = observation.lastActions[observation.agentIndex]

        if last_reward > 0:
            bandit_state[last_bandit][0] += last_reward * step
        else:
            bandit_state[last_bandit][1] += step
        
        bandit_state[observation.lastActions[0]][0] = (bandit_state[observation.lastActions[0]][0] - 1) * decay_rate + 1
        bandit_state[observation.lastActions[1]][0] = (bandit_state[observation.lastActions[1]][0] - 1) * decay_rate + 1


    if random.random() < epsilon:
      if observation.step < 1000:
        return random.choice(range(configuration["banditCount"]))
#     generate random number from Beta distribution for each agent and select the most lucky one
    best_proba = -1
    best_agent = None
    for k in range(configuration["banditCount"]):
        proba = np.random.beta(bandit_state[k][0],bandit_state[k][1])
        if proba > best_proba:
            best_proba = proba
            best_agent = k

    return best_agent

Overwriting epsilon_TS_beta.py


In [ ]:
%%writefile bayesian_ucb_TS.py

import json
import numpy as np
import pandas as pd
import random
import math
from scipy.stats import beta

bandit_state = None
total_reward = 0
epsilon = 0.1
c = 3
    
def bayesian_ucb_TS_agent (observation, configuration):
    global history, history_bandit

    step = 1.0 #you can regulate exploration / exploitation balacne using this param
    decay_rate = 0.97 # how much do we decay the win count after each call
    
    global bandit_state,total_reward, epsilon

    if observation.step == 0:
        # initial bandit state
        bandit_state = [[1,1] for i in range(configuration["banditCount"])]
    else:       
        # updating bandit_state using the result of the previous step
        last_reward = observation["reward"] - total_reward
        total_reward = observation["reward"]
        
        # we need to understand who we are Player 1 or 2
        last_bandit = observation.lastActions[observation.agentIndex]

        if last_reward > 0:
            bandit_state[last_bandit][0] += last_reward * step
        else:
            bandit_state[last_bandit][1] += step
        
        bandit_state[observation.lastActions[0]][0] = (bandit_state[observation.lastActions[0]][0] - 1) * decay_rate + 1
        bandit_state[observation.lastActions[1]][0] = (bandit_state[observation.lastActions[1]][0] - 1) * decay_rate + 1

#     generate random number from Beta distribution for each agent and select the most lucky one        
    best_proba = -1
    best_agent = None
    for k in range(configuration["banditCount"]):
        sigma = beta.std(bandit_state[k][0],bandit_state[k][1])
        count = (bandit_state[k][0]+bandit_state[k][1])
        if observation.step < 500:
          #r = np.random.beta(bandit_state[k][0],bandit_state[k][1])
          #r = bandit_state[k][0] / (bandit_state[k][0]+bandit_state[k][1])
          #sigma = beta.std(bandit_state[k][0],bandit_state[k][1])
          #count = (bandit_state[k][0]+bandit_state[k][1])
          #print("r:\t\t", r)
          #print("sigma:\t\t", sigma)
          #print("1/count:\t\t", 1/count)
          #proba = r + sigma + 1/count
          #print(proba)
          proba = bandit_state[k][0]/count + sigma * c
        #if random.random() < epsilon:
        #  proba = np.random.beta(bandit_state[k][0],bandit_state[k][1]) + beta.std(bandit_state[k][0], bandit_state[k][1]) * c
        else:
          proba = np.random.beta(bandit_state[k][0],bandit_state[k][1]) + bandit_state[k][0]/count + sigma * c
        if proba > best_proba:
            best_proba = proba
            best_agent = k

    return best_agent

Overwriting bayesian_ucb_TS.py


In [ ]:
# observation = {
#     'remainingOverageTime': 60,
#     'agentIndex': 1, # 0 or 1
#     'reward': 92, # total reward
#     'step': 184, # [0-1999]
#     'lastActions': [84, 94]
# }

# configuration:
# {'episodeSteps': 2000,
#  'actTimeout': 0.25,
#  'runTimeout': 1200,
#  'banditCount': 100,
#  'decayRate': 0.97,
#  'sampleResolution': 100}

In [2]:
%%writefile pull_same_TS_ucb.py

import numpy as np
import pandas as pd
import random, os, datetime, math
from collections import defaultdict
from scipy.stats import beta

total_reward = 0
bandit_dict = {}

def set_seed(my_seed=42):
    os.environ['PYTHONHASHSEED'] = str(my_seed)
    random.seed(my_seed)
    np.random.seed(my_seed)

def get_next_bandit():
    best_bandit = 0
    best_bandit_expected = 0
    for bnd in bandit_dict:
        # beta.std(bandit_dict[bnd]['win'], bandit_dict[bnd]['loss']*math.pow(1.03, bandit_dict[bnd]['win']+bandit_dict[bnd]['loss']+bandit_dict[bnd]['opp']))\
        expect = beta.std(bandit_dict[bnd]['win'], bandit_dict[bnd]['loss']*math.pow(1.03, bandit_dict[bnd]['win']+bandit_dict[bnd]['loss']+bandit_dict[bnd]['opp']))\
                + (bandit_dict[bnd]['win'] - bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp'] - (bandit_dict[bnd]['opp']>0)*1.5 + bandit_dict[bnd]['op_continue']) \
                / (bandit_dict[bnd]['win'] + bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp']) \
                * math.pow(0.97, bandit_dict[bnd]['win'] + bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp'])
        #print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        #print(expect)
        #print(np.random.beta(bandit_dict[bnd]['win'],bandit_dict[bnd]['loss']))
        #print(beta.std(bandit_dict[bnd]['win'],bandit_dict[bnd]['loss']) * 3)
        if expect > best_bandit_expected:
            best_bandit_expected = expect
            best_bandit = bnd
    return best_bandit

def get_next_bandit_TS():
    best_bandit = 0
    best_bandit_expected = 0
    for bnd in bandit_dict:
        # beta.std(bandit_dict[bnd]['win']*math.pow(0.97, bandit_dict[bnd]['win']+bandit_dict[bnd]['loss']+bandit_dict[bnd]['opp']), bandit_dict[bnd]['loss']) * 3 + \ 
        # + beta.std(bandit_dict[bnd]['win'],bandit_dict[bnd]['loss']) * 3 \
        expect = np.random.beta(bandit_dict[bnd]['win'], bandit_dict[bnd]['loss']) \
                + (bandit_dict[bnd]['win'] - bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp'] - (bandit_dict[bnd]['opp']>0)*1.5 + bandit_dict[bnd]['op_continue']) \
                / (bandit_dict[bnd]['win'] + bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp']) \
                * math.pow(0.97, bandit_dict[bnd]['win'] + bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp'])
        if expect > best_bandit_expected:
            best_bandit_expected = expect
            best_bandit = bnd
    return best_bandit

my_action_list = []
op_action_list = []

op_continue_cnt_dict = defaultdict(int)

def multi_armed_probabilities(observation, configuration):
    global total_reward, bandit_dict

    my_pull = 0
    if 0 == observation['step']:
        set_seed()
        total_reward = 0
        bandit_dict = {}
        for i in range(configuration['banditCount']):
            bandit_dict[i] = {'win': 1, 'loss': 1, 'opp': 0, 'my_continue': 0, 'op_continue': 0}
    else:
        last_reward = observation['reward'] - total_reward
        total_reward = observation['reward']
        
        my_idx = observation['agentIndex']
        my_last_action = observation['lastActions'][my_idx]
        op_last_action = observation['lastActions'][1-my_idx]
        
        my_action_list.append(my_last_action)
        op_action_list.append(op_last_action)
        
        if 0 < last_reward:
            bandit_dict[my_last_action]['win'] = bandit_dict[my_last_action]['win'] +1
        else:
            bandit_dict[my_last_action]['loss'] = bandit_dict[my_last_action]['loss'] +1
        bandit_dict[op_last_action]['opp'] = bandit_dict[op_last_action]['opp'] +1
        
        if observation['step'] >= 3:
            if my_action_list[-1] == my_action_list[-2]:
                bandit_dict[my_last_action]['my_continue'] += 1
            else:
                bandit_dict[my_last_action]['my_continue'] = 0
            if op_action_list[-1] == op_action_list[-2]:
                bandit_dict[op_last_action]['op_continue'] += 1
            else:
                bandit_dict[op_last_action]['op_continue'] = 0
        
        if last_reward > 0:
            my_pull = my_last_action
        else:
            if observation['step'] >= 4:
                if (my_action_list[-1] == my_action_list[-2]) and (my_action_list[-1] == my_action_list[-3]):
                    # if random.random() < 0.6:
                    # if np.random.beta(bandit_dict[my_action_list[-1]]['win'], bandit_dict[my_action_list[-1]]['loss']*math.pow(1.03, bandit_dict[my_action_list[-1]]['win']+bandit_dict[my_action_list[-1]]['loss']+bandit_dict[my_action_list[-1]]['opp'])) > 0.5:
                    if np.random.beta(bandit_dict[my_action_list[-1]]['win'], bandit_dict[my_action_list[-1]]['loss']) > 0.5:
                        my_pull = my_action_list[-1]
                    elif observation['step'] > 1000:
                        my_pull = get_next_bandit_TS()
                    else:
                        my_pull = get_next_bandit()
                elif observation['step'] > 1000:
                    my_pull = get_next_bandit_TS()
                    #if random.random() < 0.5:
                    #    my_pull = get_next_bandit()
                    #else:
                    #    my_pull = get_next_bandit_TS()
                else:
                  if (op_action_list[-1] == op_action_list[-2]) and (op_action_list[-1] == op_action_list[-3]) and (my_action_list[-1] != op_action_list[-1]):
                    # if random.random() < 0.6:
                    # if np.random.beta(bandit_dict[op_action_list[-1]]['win'], bandit_dict[op_action_list[-1]]['loss']*math.pow(1.03, bandit_dict[op_action_list[-1]]['win']+bandit_dict[op_action_list[-1]]['loss']+bandit_dict[op_action_list[-1]]['opp'])) > 0.5:
                    if np.random.beta(bandit_dict[op_action_list[-1]]['win'], bandit_dict[op_action_list[-1]]['loss']) > 0.5:
                      my_pull = op_action_list[-1]
                    else:
                      my_pull = get_next_bandit()
                  else:
                    my_pull = get_next_bandit()
            else:
                my_pull = get_next_bandit()
    
    return my_pull

Overwriting pull_same_TS_ucb.py


In [ ]:
%%writefile pull_same_TS_ucb.py

import numpy as np
import pandas as pd
import random, os, datetime, math
from collections import defaultdict
from scipy.stats import beta

total_reward = 0
bandit_dict = {}

def set_seed(my_seed=42):
    os.environ['PYTHONHASHSEED'] = str(my_seed)
    random.seed(my_seed)
    np.random.seed(my_seed)

def get_next_bandit():
    best_bandit = 0
    best_bandit_expected = 0
    for bnd in bandit_dict:
        expect = beta.std(bandit_dict[bnd]['win']*math.pow(0.97, bandit_dict[bnd]['win']+bandit_dict[bnd]['loss']+bandit_dict[bnd]['opp']), bandit_dict[bnd]['loss'])\
                + (bandit_dict[bnd]['win'] - bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp'] - (bandit_dict[bnd]['opp']>0)*1.5 + bandit_dict[bnd]['op_continue']) \
                / (bandit_dict[bnd]['win'] + bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp']) \
                * math.pow(0.97, bandit_dict[bnd]['win'] + bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp'])
        if expect > best_bandit_expected:
            best_bandit_expected = expect
            best_bandit = bnd
    return best_bandit

def get_next_bandit_TS():
    best_bandit = 0
    best_bandit_expected = 0
    for bnd in bandit_dict:
        # beta.std(bandit_dict[bnd]['win']*math.pow(0.97, bandit_dict[bnd]['win']+bandit_dict[bnd]['loss']+bandit_dict[bnd]['opp']), bandit_dict[bnd]['loss']) * 3 + \ 
        # + beta.std(bandit_dict[bnd]['win'],bandit_dict[bnd]['loss']) * 3 \
        expect = np.random.beta(bandit_dict[bnd]['win']*math.pow(0.97, bandit_dict[bnd]['win']+bandit_dict[bnd]['loss']+bandit_dict[bnd]['opp']), bandit_dict[bnd]['loss'])/2 \
                + (bandit_dict[bnd]['win'] - bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp'] - (bandit_dict[bnd]['opp']>0)*1.5 + bandit_dict[bnd]['op_continue']) \
                / (bandit_dict[bnd]['win'] + bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp']) \
                * math.pow(0.97, bandit_dict[bnd]['win'] + bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp'])
        if expect > best_bandit_expected:
            best_bandit_expected = expect
            best_bandit = bnd
    return best_bandit

my_action_list = []
op_action_list = []

op_continue_cnt_dict = defaultdict(int)

def multi_armed_probabilities(observation, configuration):
    global total_reward, bandit_dict

    my_pull = 0
    if 0 == observation['step']:
        set_seed()
        total_reward = 0
        bandit_dict = {}
        for i in range(configuration['banditCount']):
            bandit_dict[i] = {'win': 1, 'loss': 1, 'opp': 0, 'my_continue': 0, 'op_continue': 0}
    else:
        last_reward = observation['reward'] - total_reward
        total_reward = observation['reward']
        
        my_idx = observation['agentIndex']
        my_last_action = observation['lastActions'][my_idx]
        op_last_action = observation['lastActions'][1-my_idx]
        
        my_action_list.append(my_last_action)
        op_action_list.append(op_last_action)
        
        if 0 < last_reward:
            bandit_dict[my_last_action]['win'] = bandit_dict[my_last_action]['win'] +1
        else:
            bandit_dict[my_last_action]['loss'] = bandit_dict[my_last_action]['loss'] +1
        bandit_dict[op_last_action]['opp'] = bandit_dict[op_last_action]['opp'] +1
        
        if observation['step'] >= 3:
            if my_action_list[-1] == my_action_list[-2]:
                bandit_dict[my_last_action]['my_continue'] += 1
            else:
                bandit_dict[my_last_action]['my_continue'] = 0
            if op_action_list[-1] == op_action_list[-2]:
                bandit_dict[op_last_action]['op_continue'] += 1
            else:
                bandit_dict[op_last_action]['op_continue'] = 0
        
        if last_reward > 0:
            my_pull = my_last_action
        else:
            if observation['step'] >= 4:
              if observation['step'] < 1000:
                if (my_action_list[-1] == my_action_list[-2]) and (my_action_list[-1] == my_action_list[-3]):
                    if np.random.beta(bandit_dict[my_action_list[-1]]['win']*math.pow(0.97, bandit_dict[my_action_list[-1]]['win']+bandit_dict[my_action_list[-1]]['loss']+bandit_dict[my_action_list[-1]]['opp']), bandit_dict[my_action_list[-1]]['loss']) > 0.4:
                        my_pull = my_action_list[-1]
                    else:
                        my_pull = get_next_bandit()
                #elif (op_action_list[-1] == op_action_list[-2]) and (op_action_list[-1] == op_action_list[-3]) and (my_action_list[-1] != op_action_list[-1]):
                #    if np.random.beta(bandit_dict[op_action_list[-1]]['win']*math.pow(0.97, bandit_dict[op_action_list[-1]]['win']+bandit_dict[op_action_list[-1]]['loss']+bandit_dict[op_action_list[-1]]['opp']), bandit_dict[op_action_list[-1]]['loss']) > 0.6:
                #      my_pull = op_action_list[-1]
                #    else:
                #      my_pull = get_next_bandit()
                else:
                    my_pull = get_next_bandit()
              else:
                my_pull = get_next_bandit_TS()
            else:
                my_pull = get_next_bandit()
    
    return my_pull

Overwriting pull_same_TS_ucb.py


In [ ]:
!pip install kaggle-environments --upgrade -q

     |████████████████████████████████| 112kB 7.1MB/s 
     |████████████████████████████████| 61kB 5.1MB/s 
ERROR: nbclient 0.5.1 has requirement jupyter-client>=6.1.5, but you'll have jupyter-client 5.3.5 which is incompatible.


In [ ]:
from kaggle_environments import make

env = make("mab", debug=True)

Loading environment football failed: No module named 'gfootball'


In [ ]:
!ls

baby-steps-of-rl-ja	    epsilon_TS_gamma.py  submission.py
bayesian_ucb.py		    experiment.py	 thompson_sampling.py
bayesian_ucb_TS_epsilon.py  Kaggle_bandit.ipynb  ucb_decay.py
bayesian_ucb_TS.py	    pull_same.py
epsilon_TS_beta.py	    RL_Python.ipynb


In [ ]:
env.reset()
# env.run(["experiment.py", "bayesian_ucb.py"])
# env.run(["experiment.py", "thompson_sampling.py"])
#env.run(["epsilon_TS_beta.py", "bayesian_ucb.py"])
#env.run(["epsilon_TS_beta.py", "thompson_sampling.py"])
#env.run(["epsilon_TS_gamma.py", "thompson_sampling.py"])
#env.run(["bayesian_ucb_TS.py", "bayesian_ucb.py"])
#env.run(["bayesian_ucb_TS_epsilon.py", "bayesian_ucb.py"])
#env.run(["bayesian_ucb_TS.py", "thompson_sampling.py"])
#env.run(["bayesian_ucb_TS.py", "epsilon_TS_beta.py"])
#env.run(["pull_same.py", "bayesian_ucb_TS.py"])
env.run(["pull_same_TS_ucb.py", "pull_same.py"])
env.render(mode="ipython", width=800, height=500)

Output hidden; open in https://colab.research.google.com to view.

＊実験失敗

In [ ]:
%%writefile bayesian_ucb_TS_epsilon.py

import json
import numpy as np
import pandas as pd
import random
import math
from scipy.stats import beta

bandit_state = None
total_reward = 0
epsilon = 0.12
c = 3
    
def bayesian_ucb_TS_agent (observation, configuration):
    global history, history_bandit

    step = 1.0 #you can regulate exploration / exploitation balacne using this param
    decay_rate = 0.97 # how much do we decay the win count after each call
    
    global bandit_state,total_reward, epsilon

    if observation.step == 0:
        # initial bandit state
        bandit_state = [[1,1] for i in range(configuration["banditCount"])]
    else:       
        # updating bandit_state using the result of the previous step
        last_reward = observation["reward"] - total_reward
        total_reward = observation["reward"]
        
        # we need to understand who we are Player 1 or 2
        last_bandit = observation.lastActions[observation.agentIndex]

        if last_reward > 0:
            bandit_state[last_bandit][0] += last_reward * step
        else:
            bandit_state[last_bandit][1] += step
        
        bandit_state[observation.lastActions[0]][0] = (bandit_state[observation.lastActions[0]][0] - 1) * decay_rate + 1
        bandit_state[observation.lastActions[1]][0] = (bandit_state[observation.lastActions[1]][0] - 1) * decay_rate + 1

#     generate random number from Beta distribution for each agent and select the most lucky one   
    if random.random() < epsilon:
      if observation.step < 1000:
        return random.choice(range(configuration["banditCount"]))
    best_proba = -1
    best_agent = None
    for k in range(configuration["banditCount"]):
        if observation.step < 500:
          proba = bandit_state[k][0] / (bandit_state[k][0]+bandit_state[k][1]) + beta.std(bandit_state[k][0],bandit_state[k][1]) * c
        else:
          proba = np.random.beta(bandit_state[k][0],bandit_state[k][1])
        if proba > best_proba:
            best_proba = proba
            best_agent = k

    return best_agent

Overwriting bayesian_ucb_TS_epsilon.py


In [ ]:
%%writefile epsilon_TS_gamma.py

import json
import numpy as np
import pandas as pd
import random

bandit_state = None
total_reward = 0
epsilon = 0.1
    
def epsilon_TS_gamma_agent (observation, configuration):
    global history, history_bandit

    step = 1.0 #you can regulate exploration / exploitation balacne using this param
    decay_rate = 0.97 # how much do we decay the win count after each call
    
    global bandit_state,total_reward, epsilon

    if observation.step == 0:
        # initial bandit state
        bandit_state = [[1,1] for i in range(configuration["banditCount"])]
    else:       
        # updating bandit_state using the result of the previous step
        last_reward = observation["reward"] - total_reward
        total_reward = observation["reward"]
        
        # we need to understand who we are Player 1 or 2
        last_bandit = observation.lastActions[observation.agentIndex]

        if last_reward > 0:
            bandit_state[last_bandit][0] += last_reward * step
        else:
            bandit_state[last_bandit][1] += step
        
        bandit_state[observation.lastActions[0]][0] = (bandit_state[observation.lastActions[0]][0] - 1) * decay_rate + 1
        bandit_state[observation.lastActions[1]][0] = (bandit_state[observation.lastActions[1]][0] - 1) * decay_rate + 1


    if random.random() < epsilon:
      if observation.step < 1000:
        return random.choice(range(configuration["banditCount"]))
#     generate random number from Beta distribution for each agent and select the most lucky one        
    best_proba = -1
    best_agent = None
    for k in range(configuration["banditCount"]):
        proba = np.random.gamma(bandit_state[k][0],bandit_state[k][1])
        if proba > best_proba:
            best_proba = proba
            best_agent = k

    return best_agent

Writing epsilon_TS_gamma.py


In [ ]:
%%writefile experiment.py

import math
import numpy as np
from scipy.stats import beta

win, lose, bandit, root = [None] * 4
total_reward = 0
c = 3

def agent(observation, configuration):
    global total_reward, bandit, win, lose, c, root

    if observation.step == 0:
        win, lose, root = np.ones((3, configuration.banditCount))
    else:
        r = observation.reward - total_reward
        total_reward = observation.reward
        # Update Gaussian posterior
        win[bandit] += r
        lose[bandit] += 1 - r
        root[bandit] = math.sqrt(win[bandit]+lose[bandit])

    bound = win / (win + lose) + beta.std(win, lose) * c / root
    bandit = int(np.argmax(bound))
    
    return bandit

Overwriting experiment.py


＊初期設定のテスト

In [ ]:
!ls

In [ ]:
%%writefile pull_same.py

import numpy as np
import pandas as pd
import random, os, datetime, math
from collections import defaultdict

total_reward = 0
bandit_dict = {}

def set_seed(my_seed=42):
    os.environ['PYTHONHASHSEED'] = str(my_seed)
    random.seed(my_seed)
    np.random.seed(my_seed)

def get_next_bandit():
    best_bandit = 0
    best_bandit_expected = 0
    for bnd in bandit_dict:
        expect = (bandit_dict[bnd]['win'] - bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp'] - (bandit_dict[bnd]['opp']>0)*1.5 + bandit_dict[bnd]['op_continue']) \
                 / (bandit_dict[bnd]['win'] + bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp']) \
                * math.pow(0.97, bandit_dict[bnd]['win'] + bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp'])
        #print("1:\t", expect)
        a = bandit_dict[bnd]['win'] - bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp'] - (bandit_dict[bnd]['opp']>0)*1.5 + bandit_dict[bnd]['op_continue']
        b = bandit_dict[bnd]['win'] + bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp']
        c = math.pow(0.97, bandit_dict[bnd]['win'] + bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp'])
        #print("2:\t", a+a/b*c)
        #print("3:\t", (a/b)*c)
        #print("3:\t", a+(a/b*c))
        if expect > best_bandit_expected:
            best_bandit_expected = expect
            best_bandit = bnd
    return best_bandit

my_action_list = []
op_action_list = []

op_continue_cnt_dict = defaultdict(int)

def multi_armed_probabilities(observation, configuration):
    global total_reward, bandit_dict

    my_pull = random.randrange(configuration['banditCount'])
    if 0 == observation['step']:
        set_seed()
        total_reward = 0
        bandit_dict = {}
        for i in range(configuration['banditCount']):
            bandit_dict[i] = {'win': 1, 'loss': 0, 'opp': 0, 'my_continue': 0, 'op_continue': 0}
    else:
        last_reward = observation['reward'] - total_reward
        total_reward = observation['reward']
        
        my_idx = observation['agentIndex']
        my_last_action = observation['lastActions'][my_idx]
        op_last_action = observation['lastActions'][1-my_idx]
        
        my_action_list.append(my_last_action)
        op_action_list.append(op_last_action)
        
        if 0 < last_reward:
            bandit_dict[my_last_action]['win'] = bandit_dict[my_last_action]['win'] +1
        else:
            bandit_dict[my_last_action]['loss'] = bandit_dict[my_last_action]['loss'] +1
        bandit_dict[op_last_action]['opp'] = bandit_dict[op_last_action]['opp'] +1
        
        if observation['step'] >= 3:
            if my_action_list[-1] == my_action_list[-2]:
                bandit_dict[my_last_action]['my_continue'] += 1
            else:
                bandit_dict[my_last_action]['my_continue'] = 0
            if op_action_list[-1] == op_action_list[-2]:
                bandit_dict[op_last_action]['op_continue'] += 1
            else:
                bandit_dict[op_last_action]['op_continue'] = 0
        
        if last_reward > 0:
            my_pull = my_last_action
        else:
            if observation['step'] >= 4:
                if (my_action_list[-1] == my_action_list[-2]) and (my_action_list[-1] == my_action_list[-3]):
                    if random.random() < 0.5:
                        my_pull = my_action_list[-1]
                    else:
                        my_pull = get_next_bandit()
                else:
                    my_pull = get_next_bandit()
            else:
                my_pull = get_next_bandit()
    
    return my_pull

Overwriting pull_same.py


In [ ]:
%%writefile ucb_decay.py

import numpy as np

decay = 0.97
total_reward = 0
bandit = None

def agent(observation, configuration):
    #print(observation)
    #if observation.step == 100:
    #  print(float(observation["reward"]))
    #  print(observation["lastActions"][1-observation.agentIndex])      
    if observation.step == 0:
    #  print("------------test---------------")
    #  print(observation)
      print(configuration)
    #if observation["step"] == 0:
    #  print("------------test---------------")
    global reward_sums, n_selections, total_reward, bandit
    
    n_bandits = configuration.banditCount

    if observation.step == 0:
        n_selections, reward_sums = np.full((2, n_bandits), 1e-32)
    else:
        #print(observation.step, ": ", observation.reward - total_reward)
        reward_sums[bandit] += decay * (observation.reward - total_reward)
        total_reward = observation.reward

    avg_reward = reward_sums / n_selections    
    delta_i = np.sqrt(2 * np.log(observation.step + 1) / n_selections)
    bandit = int(np.argmax(avg_reward + delta_i))

    n_selections[bandit] += 1

    return bandit

Overwriting ucb_decay.py


In [ ]:
%%writefile bayesian_ucb.py

import numpy as np
from scipy.stats import beta

post_a, post_b, bandit = [None] * 3
total_reward = 0
c = 3

def agent(observation, configuration):
    global total_reward, bandit, post_a, post_b, c

    if observation.step == 0:
        post_a, post_b = np.ones((2, configuration.banditCount))
    else:
        r = observation.reward - total_reward
        total_reward = observation.reward
        # Update Gaussian posterior
        post_a[bandit] += r
        post_b[bandit] += 1 - r
    
    bound = post_a / (post_a + post_b) + beta.std(post_a, post_b) * c
    bandit = int(np.argmax(bound))
    
    return bandit

Overwriting bayesian_ucb.py


In [ ]:
%%writefile thompson_sampling.py

import json
import numpy as np
import pandas as pd

bandit_state = None
total_reward = 0
last_step = None
    
def multi_armed_bandit_agent (observation, configuration):
    global history, history_bandit

    step = 1.0 #you can regulate exploration / exploitation balacne using this param
    decay_rate = 0.97 # how much do we decay the win count after each call
    
    global bandit_state,total_reward,last_step
        
    if observation.step == 0:
        # initial bandit state
        bandit_state = [[1,1] for i in range(configuration["banditCount"])]
    else:       
        # updating bandit_state using the result of the previous step
        last_reward = observation["reward"] - total_reward
        total_reward = observation["reward"]
        
        # we need to understand who we are Player 1 or 2
        player = int(last_step == observation.lastActions[1])
        
        if last_reward > 0:
            bandit_state[observation.lastActions[player]][0] += last_reward * step
        else:
            bandit_state[observation.lastActions[player]][1] += step
        
        bandit_state[observation.lastActions[0]][0] = (bandit_state[observation.lastActions[0]][0] - 1) * decay_rate + 1
        bandit_state[observation.lastActions[1]][0] = (bandit_state[observation.lastActions[1]][0] - 1) * decay_rate + 1

#     generate random number from Beta distribution for each agent and select the most lucky one
    best_proba = -1
    best_agent = None
    for k in range(configuration["banditCount"]):
        proba = np.random.beta(bandit_state[k][0],bandit_state[k][1])
        if proba > best_proba:
            best_proba = proba
            best_agent = k
        
    last_step = best_agent
    return best_agent

In [ ]:
from kaggle_environments import make

env = make("mab", debug=True)

In [ ]:
env.reset()
#env.run(["ucb_decay.py", "bayesian_ucb.py"])
env.run(["bayesian_ucb.py", "thompson_sampling.py"])
env.render(mode="ipython", width=800, height=500)

Output hidden; open in https://colab.research.google.com to view.

＊初期設定

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

In [ ]:
# download API key from google drive
## Original: https://colab.research.google.com/drive/1eufc8aNCdjHbrBhuy7M7X6BGyzAyRbrF#scrollTo=y5_288BYp6H1
## When you run for the first time, you will see a link to authenticate.

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)